## Model IO
모델 I/O 모듈에는 입력(input)과 출력(output)이 있음.
입력과 출력의 뜻으로 input은 우선 prompt(명령 등을 내리는 곳)임. (Prompts, Language models, Output parser 등이 있음.)

## Retrueval (리트리블)
- Retrueval은 `외부 데이터`로 접근하여 이를 모델에 `어떻게 제공`하는 것에 관한 거임.
- document loaders, Transformers, text embedding , vector stores , retrievers 등이 있음.
- 이것들 전부 우리의 데이터들로 작업하게 하고 모델에게 제공할 수 있는지에 관한 거임. ( 대충 외부 데이터를 가지고 모델과 작업하는 부분 )

## Chains
- 이전에 했던 내용. ( 나중에 자세히 작성 )

## Agents 
- `독립적으로 AI가 작동하도록 만들 수 있게 해주는 agents`.
- chains이 필요한 도구들을 선택하여 사용할 수 있도록 해줌.
- 그냥 chains에게 일을 맞기고 그에 맞는 커스텀 도구를 만들어 준다면 chains 스스로 사용할 tool들을 선택함...(?)

## Memory (기억)
- 챗봇에 Memory(기억)할 수 있도록 하는 것.

## Callbacks 
- callbacks는 기본적으로 model이 무엇을 하고 있는지, 중간에 알 수 있도록 하는 것. ( intermediate )
- 모델이 답변을 제공하기 전에 모델이 어떤 일을 하고 있는지 확인 할 수 있음. (무엇을 생각하고 있는지.)

In [1]:
# Template 처리.

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

template = PromptTemplate.from_template("What is the capital of {country}")

template.format(country="France")
# template.format() # <- error(필수 적인 값 France이 없기 떄문)

template = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"]
) # 이렇게도 가능.

template.format(country="France")

'What is the capital of France'

In [14]:
# FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
# 미리 대답해 놓은 예시
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
    },
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
# example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}") ## same

# 랭체인이 알아서 각각의 예제 리스트들을 prompt를 사용해서 형식화 할꺼임.
# question , answer 값으로 -> `이건 다르게 넣으면 안됨!`
# suffix는 형식화가 끝난 후 마지막에 나오는 거임.
# suffix에서 어떤 변수를 사용해야하는지 input_variables에 작성해 줘야함.
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

# prompt.format(country="Korea")
# 이렇게 해주면, 이제 어떻게 대답해야 하는지에 대한 형식을 알아서 형식에 맞게 대답해줌.! 대충 예제를 가져오고 예제를 prompt로 형식화 하고( 오브젝트 이름에 맞게 설정! example_template 참조.. ) 
# 그리고 형식화 된 값을 FewShot에게 전달해주기만 해도 양식에 맞춰서 대답해줌.. good..
chain = prompt | chat

chain.invoke({
    "country" : "Korea"
})

AI: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='AI: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

## FewShotPromptTemplate , Fewshot Learning
- `Fewshot은 모델에게 예제를 준다는 뜻과 같음.`
- 더 나은 대답을 할 수 있도록 하는 예제를 줄 수 있음.
- 복잡한 대답을 원할 경우 모델에게 `어떻게 대답해야 하는 지에 대한 예제를 AI모델에게 주는게(Fewshot) prompt를 사용해서 어떻게 대답해야 하는지 알려주는 것보다 훨씬 좋음.`
- 예를 들어 모델에게 콤마(,)를 써서 구분해줘, 소문자만 써야해 등... ( 모델이 text를 생성하기에 어떻게 대답해야 하는지에 대한 예시를 미리 주는게 더 좋은 방식임! `생성하기 전에 예시를 줌!` 생성 할 경우에 주는게 아니라! )
- 만약 고객지원 봇을 만들때 이전 데이터( 이전에 운영 했을 경우 )가 있을 경우 많은 고객들과 대화 기록이 남아 있을 것임. 그럼 language model에게 어떻게 말해야 할지(대응 해야할지)에 대해 알려주고 싶을꺼임.(많은 데이터가 있기 때문에) 그럼 그냥 대화 내용을 가져와서 형식화 시켜주면 더 잘 만들 수 있을것임.
- `FewShotPromptTemplate`은 형식을 주고 그 형식대로 만들어 돌라는 것을 하는거임! 엄청난 기능!
